# Workbook for optimizing foci segmentation thresholds

This workbook provides step-by-step instructions for determining segmentation thresholds for identifying foci within cells. __Do not edit this workbook file directly!__ Make a copy for each optimization that you perform and edit that version. Save this template in an unedited form for future experiments.

This workbook uses a built-in "optimization mode" that only pulls the first three images out of multi-image .czi files to dramatically speed up processing. This will allow you to tinker with thresholds and find the ideal cutoffs.

I found that the most effective way to define these cutoffs was to use a positive control image - one where there were many foci - and a negative control image with few to no foci to set the thresholds. This allows you to identify cutoffs that simultaneously maximize sensitivity while minimizing false positives.

__Before running this notebook, you must have completed the setup notebook in the install folder.__

In [ ]:
# import required code
from csth_analysis import find_cells, segment_cells, foci
import os

The cell below defines the file paths for the files you will need during this process. Change each variable as needed.

In [ ]:
neg_ctrl_czi = '/path/to/neg/ctrl/file.czi'
pos_ctrl_czi = '/path/to/pos/ctrl/file.czi'
neg_empty_field_czi = '/path/to/neg/empty/field/file.czi'
pos_empty_field_czi = '/path/to/pos/empty/field/file.czi'
svm_pkl = '/path/to/csth-imaging/trained_svm.pkl' # change /path/to/ so it points to the csth-imaging folder

Now that you have defined the important variables, you will perform pre-processing. This takes the following steps:
1. Load images
2. Identify regions of the field containing cells
3. Identify nuclei
4. Segment cells  
    _Note:_ By default this segments cells using the 488 wavelength. Change 488 to your desired wavelength in the splitter.segment_cells() command to change wavelengths.
5. Initialize foci detection object.

The steps are indicated in the code blocks.

In [ ]:
## NOTE: RUNNING THIS CELL WILL TAKE A WHILE! DON'T BE CONCERNED! ##

# 1. load images and the trained SVM focus classifier, and 2.
neg_ctrl_finder = find_cells.MultiFinder(filename=neg_ctrl_czi,
                                         bg_filename=empty_field_czi,
                                         oof_svm=svm_pkl,
                                         optim=True)
pos_ctrl_finder = find_cells.MultiFinder(filename=pos_ctrl_czi,
                                         bg_filename=empty_field_czi,
                                         oof_svm=svm_pkl,
                                         optim=True)
print('MultiFinder created.')

# 2. find cells
neg_ctrl_splitter = segment_cells.CellSplitter(neg_ctrl_finder)
pos_ctrl_splitter = segment_cells.CellSplitter(pos_ctrl_finder)

# 3. identify nuclei
neg_ctrl_splitter.segment_nuclei(verbose=True)
pos_ctrl_splitter.segment_nuclei(verbose=True)

# 4. Segment cells.
neg_ctrl_splitter.segment_cells(488, verbose=True)
pos_ctrl_splitter.segment_cells(488, verbose=True)

# 5. Initialize foci detection objects.
neg_foci = foci.Foci(neg_ctrl_splitter, verbose=True)
pos_foci = foci.Foci(pos_ctrl_splitter, verbose=True)

Next, you're ready to try segmenting foci and checking the output to see if it looks correct.

A bit of background on how Canny edge detection works so you can logically select values. There are two threshold numbers that go into detecting foci:
- high threshold: the minimum edge intensity to _seed_ a new object. The entire object doesn't have to have an edge this sharp, but if no part of it does, it won't be identified.
- low threshold: the minimum edge intensity to _grow_ an object. If one point already satisfied the high threshold, it can grow even if it has a dimmer edge as long as that dimmer edge is above the low threshold.  

If this doesn't make sense, read about the Canny edge detection algorithm on Wikipedia or elsewhere.

I often start with values of 10,000 and 7,500 for the high and low thresholds respectively, so those are the default values below.

Running the code block below will perform segmentation and save the raw images as well as the segmented foci to the output_dir you select in the first line. You can then open these images in fiji, overlay the segmented foci onto the raw images, and determine if the threshold is too high/too low. The code will also output some descriptive information about the number of dots, their intensities, etc. which may be useful to you in deciding thresholds.

In [ ]:
from skimage import io  # for saving images
output_dir = '/path/to/images/'  # change this to where you'd like the output images to go.
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
# save raw images
for c in pos_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, pos_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'pos_raw_' + str(c) + '_' + str(im) + '.tif',
                 pos_foci.imgs[c][im, :, :, :])
for c in neg_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, neg_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'neg_raw_' + str(c) + '_' + str(im) + '.tif',
                 neg_foci.imgs[c][im, :, :, :])        

## The following cell is for running segmentation on both channels.

In [ ]:
# edit the (high, low) threshold pairs below to optimize segmentation.
green_thresholds = (10000, 7500) 
red_thresholds = (10000, 7500)
# perform segmentation
neg_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds})
pos_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds})
# save segmentation outputs
for c in pos_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(pos_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'pos_foci_' + str(c) + '_' + str(im) + '.tif',
                 pos_foci.foci[c][im])
for c in neg_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(neg_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'neg_foci_' + str(c) + '_' + str(im) + '.tif',
                 neg_foci.foci[c][im])

Edit the thresholds in the cell above and re-run as needed until you reach ideal thresholds.

## The following cell is for running segmentation on just one channel.

In [ ]:
desired_channel = 488  # change this to 488 for green or 561 for red
# edit the (high, low) threshold pair below to optimize segmentation.
thresholds = (10000, 7500) 
# perform segmentation
neg_foci.segment(verbose=True, thresholds={desired_channel: thresholds},
                 seg_channels=(desired_channel))
pos_foci.segment(verbose=True, thresholds={desired_channel: thresholds},
                 seg_channels=(desired_channel))
# save segmentation outputs
for im in range(0, len(pos_foci.foci[desired_channel])):  # for each image in that channel
    io.imsave(output_dir + 'pos_foci_' + str(desired_channel) + '_' + str(im) + '.tif',
             pos_foci.foci[desired_channel][im])
for im in range(0, len(neg_foci.foci[desired_channel])):  # for each image in that channel
    io.imsave(output_dir + 'neg_foci_' + str(desired_channel) + '_' + str(im) + '.tif',
             neg_foci.foci[desired_channel][im])